In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd

from dhdrnet.util import DATA_DIR, ROOT_DIR

figdir = ROOT_DIR / "figures"

In [45]:
import re

df = (
    pd.read_csv(
        ROOT_DIR / "precomputed_data" / "store_current.csv", index_col=0
    )
    .drop(columns="ev1")
    .rename(columns={"ev2": "ev"})
)

p = re.compile(r"([0-9.]+)")
mask = df["metric"] == "perceptual"
df.loc[mask, "score"] = df.loc[mask, "score"].transform(lambda s: p.search(s).group(0))
df["score"] = pd.to_numeric(df["score"])
df

,name,metric,ev,score
0,c1b1_20150312_141022_171,rmse,-3.00,0.814936
1,c1b1_20150312_141022_171,psnr,-3.00,6.552593
2,c1b1_20150312_141022_171,ssim,-3.00,0.232993
3,c1b1_20150312_141022_171,perceptual,-3.00,0.443095
4,c1b1_20150312_141022_171,rmse,-2.75,0.816721
...,...,...,...,...
139,5066_20160510_193314_625,perceptual,5.75,0.091892
140,5066_20160510_193314_625,rmse,6.00,0.258454
141,5066_20160510_193314_625,psnr,6.00,17.143010
142,5066_20160510_193314_625,ssim,6.00,0.845857


In [55]:
grouped = df.set_index("name").groupby(["metric"])
grouped.describe()["score"][["mean", "std", "min", "max"]]

,mean,std,min,max
metric,,,,
perceptual,0.183580,0.146395,0.001431,0.853125
psnr,16.431658,6.014727,5.739877,47.463439
rmse,0.354830,0.235042,0.007798,0.991846
ssim,0.724803,0.267346,0.015141,0.997507


In [52]:
grouped.min()["score"]

metric
perceptual    0.001431
psnr          5.739877
rmse          0.007798
ssim          0.015141
Name: score, dtype: float64

In [9]:
ind_minscore["5066_20160416_100212_436"]

metric
psnr    16
rmse    72
ssim    17
Name: score, dtype: int64

In [20]:
df2 = pd.read_csv(ROOT_DIR / "precomputed_data" / "store_no_synth.csv")
df2 = df2[(df2["ev1"] == 0) | (df2["ev2"] == 0)]

In [21]:
df2["ev"] = df2[["ev1", "ev2"]].apply(
    lambda evs: [e for e in evs if e != 0][0], axis=1
)
df2 = df2.drop(columns=["ev1", "ev2"])
df2

,name,metric,score,ev
33,0009_20160703_160626_325,mse,344.949785,-3.00
34,0009_20160703_160626_325,ssim,0.951258,-3.00
35,0009_20160703_160626_325,perceptual,0.049351,-3.00
117,0009_20160703_160626_325,mse,353.442578,-2.75
118,0009_20160703_160626_325,ssim,0.951578,-2.75
...,...,...,...,...
7230337,33TJ_20150730_201700_327,ssim,0.808433,5.75
7230338,33TJ_20150730_201700_327,perceptual,0.161453,5.75
7230339,33TJ_20150730_201700_327,mse,3017.795220,6.00
7230340,33TJ_20150730_201700_327,ssim,0.789549,6.00


In [32]:
import numpy as np
rmse = df2[df2["metric"] == "mse"]
rmse = rmse["score"].apply(np.sqrt)
rmse.max()

131.68803635158815

In [33]:
df[df["metric"] == "rmse"]

name      c483_20150901_211303_855
metric                        rmse
ev                               6
score                      1.14268
dtype: object